# Machine Learning Project : Tax Risk Identification
Gabriel SULTAN - Lalith Adithya CHANUMOLU - DIA 6

We'll be using the Tax Risk Identification Dataset available on Kaggle here : https://www.kaggle.com/datasets/ziya07/tax-risk-identification-dataset


